<a href="https://colab.research.google.com/github/stephenkati/Kiswahili-Mozilla-Common-Voice/blob/main/Kiswahili_NLP_ASR%2C_Translation_%26_Zero_Shot_Text_Classification_with_Hugging_Face.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!wget https://radiodownloaddw-a.akamaihd.net/downloads/Events/dwelle/dira/mp3/kis/E6F773D5_2.mp3
!ffmpeg -i  E6F773D5_2.mp3 -ar 16000 -ac 1 -f wav football.wav

--2023-10-21 08:31:32--  https://radiodownloaddw-a.akamaihd.net/downloads/Events/dwelle/dira/mp3/kis/E6F773D5_2.mp3
Resolving radiodownloaddw-a.akamaihd.net (radiodownloaddw-a.akamaihd.net)... 23.200.156.210, 23.200.156.215
Connecting to radiodownloaddw-a.akamaihd.net (radiodownloaddw-a.akamaihd.net)|23.200.156.210|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2951943 (2.8M) [audio/mpeg]
Saving to: ‘E6F773D5_2.mp3’

E6F773D5_2.mp3      100%[===================>]   2.81M  --.-KB/s    in 0.08s   

2023-10-21 08:31:32 (35.1 MB/s) - ‘E6F773D5_2.mp3’ saved [2951943/2951943]

ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-lib

In [2]:
!pip install transformers
!pip install SoundFile

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 42.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 29.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 62.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 42.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 26.8 MB/s eta 0:00:00


In [ ]:
import soundfile as sf
import torch
from transformers import Wav2Vec2ForCTC, Wav2Vec2Processor
import argparse

def parse_transcription(wav_file):
    # load pretrained model
    processor = Wav2Vec2Processor.from_pretrained("alokmatta/wav2vec2-large-xlsr-53-sw")
    model = Wav2Vec2ForCTC.from_pretrained("alokmatta/wav2vec2-large-xlsr-53-sw")

    # load audio
    audio_input, sample_rate = sf.read(wav_file)

    # pad input values and return pt tensor
    input_values = processor(audio_input, sampling_rate=sample_rate, return_tensors="pt").input_values

    # INFERENCE
    # retrieve logits & take argmax
    logits = model(input_values).logits
    predicted_ids = torch.argmax(logits, dim=-1)

    # transcribe
    transcription = processor.decode(predicted_ids[0], skip_special_tokens=True)
    return transcription

parse_transcription("football.wav")

/usr/local/lib/python3.10/dist-packages/transformers/configuration_utils.py:380: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(


Some weights of the model checkpoint at alokmatta/wav2vec2-large-xlsr-53-sw were not used when initializing Wav2Vec2ForCTC: ['wav2vec2.encoder.pos_conv_embed.conv.weight_g', 'wav2vec2.encoder.pos_conv_embed.conv.weight_v']
- This IS expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at alokmatta/wav2vec2-large-xlsr-53-sw and are newly initialized: ['wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original0', 'wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original1']
You should probably T

In [1]:
!pip install pydub

In [3]:
from pydub import AudioSegment
import soundfile as sf  # Add this import
import torch
from transformers import Wav2Vec2ForCTC, Wav2Vec2Processor

def transcribe_audio(audio_file):
    processor = Wav2Vec2Processor.from_pretrained("alokmatta/wav2vec2-large-xlsr-53-sw")
    model = Wav2Vec2ForCTC.from_pretrained("alokmatta/wav2vec2-large-xlsr-53-sw")

    chunk_size = 10  # seconds (adjust as needed)

    audio = AudioSegment.from_file(audio_file)

    for start in range(0, len(audio), chunk_size * 1000):  # Convert to milliseconds
        chunk = audio[start:start + chunk_size * 1000]
        chunk.export("temp.wav", format="wav")  # Export the chunk as a temporary WAV file

        # load audio
        audio_input, sample_rate = sf.read("temp.wav")

        # pad input values and return pt tensor
        input_values = processor(audio_input, sampling_rate=sample_rate, return_tensors="pt").input_values

        # INFERENCE
        # retrieve logits & take argmax
        logits = model(input_values).logits
        predicted_ids = torch.argmax(logits, dim=-1)

        # transcribe
        transcription = processor.decode(predicted_ids[0], skip_special_tokens=True)
        print("Transcription:", transcription)

# Call the function with your audio file
transcribe_audio("football.wav")


Some weights of the model checkpoint at alokmatta/wav2vec2-large-xlsr-53-sw were not used when initializing Wav2Vec2ForCTC: ['wav2vec2.encoder.pos_conv_embed.conv.weight_g', 'wav2vec2.encoder.pos_conv_embed.conv.weight_v']
- This IS expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at alokmatta/wav2vec2-large-xlsr-53-sw and are newly initialized: ['wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original0', 'wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original1']
You should probably T

Transcription: ni mchezo ambao kwa kweli ulikuwa ni mzuri sana na timu zote zimejitahidi kad
Transcription: driya uwezo wake e kuweza kupata ushindi hasa abricost ambayo ilikuwa imecheza kwa e
Transcription: e kwa nia hasa ya kupata ushindi kabla ya lakika tisimi au mia naishirini e lakini misri walikuwa wana
Transcription: jihami kidogo na kushambulia hapa na pale na hiyo imewasaidia isri kwamba wamesika mpakao
Transcription: e dakika mia na ishirini zimekwenda na pendate e haina mwenyewe wameweza kue kupata ushindi dini yaikweta iliweza kuiondoga
Transcription: wa mali ambao mali ni moja wapo ya etimu ambayo tunaesema ni taifa kubwa wkisoka barani afrika ulitarajia hilo yalitarajia
Transcription: kwa sababu e timu ya mali haipokatika tiwango kile cha miaka kama ishirini e iliopita na
Transcription: guinea ikweta e ni pamoja na timu hizi zote ndogo komoro kwa mfano e halafu ukiangalia gambia kwa mfanoe
Transcription: zote hizi ni timu ndogo lakini zimechachafia sana katika mashindano 